In [0]:
!pip install Penman

In [0]:
!python -V

Python 3.6.7


In [0]:
!wget -O - https://debian.neo4j.org/neotechnology.gpg.key | sudo apt-key add -
!echo 'deb https://debian.neo4j.org/repo stable/' | sudo tee -a /etc/apt/sources.list.d/neo4j.list
!sudo apt-get update
!apt-get install neo4j=1:3.5.1
!neo4j-admin set-initial-password root
!service neo4j start
!pip install py2neo

--2019-02-18 05:01:53--  https://debian.neo4j.org/neotechnology.gpg.key
Resolving debian.neo4j.org (debian.neo4j.org)... 35.169.240.221, 52.1.29.249
Connecting to debian.neo4j.org (debian.neo4j.org)|35.169.240.221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4791 (4.7K) [application/octet-stream]
Saving to: ‘STDOUT’

-                   100%[===================>]   4.68K  --.-KB/s    in 0s      

2019-02-18 05:01:53 (840 MB/s) - written to stdout [4791/4791]

OK
deb https://debian.neo4j.org/repo stable/
Ign:1 https://debian.neo4j.org/repo stable/ InRelease
Get:2 https://debian.neo4j.org/repo stable/ Release [1,479 B]
Get:3 https://debian.neo4j.org/repo stable/ Release.gpg [819 B]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,609 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-s

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
filename = '/content/drive/My Drive/Colab_Folders/AynRandTheFountainheadPl.txt'
file = open(filename, 'rt')
text = file.read()
file.close()

In [0]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
from nltk import sent_tokenize
sentences = sent_tokenize(text)
print(sentences[35:37])

['I had been opposed to the practice of dedicating books; I had held that a book is addressed to any reader who proves worthy of it.', 'But, that night, I told Frank that I would dedicate The Fountainhead to him because he had saved it.']


In [0]:
list_str = sentences[30:82]
print(list_str)

["I gave that line to Dominique at Roark's trial.", 'I did not feel discouragement very often, and when I did, it did not last longer than overnight.', 'But there was one evening, during the writing of The Fountainhead, when I felt so profound an indignation at the state of “things as they are” that it seemed as if I would never regain the energy to move one step farther toward “things as they ought to be.” Frank talked to me for hours, that night.', 'He convinced me of why one cannot give up the world to those one despises.', 'By the time he finished, my discouragement was gone; it never came back in so intense a form.', 'I had been opposed to the practice of dedicating books; I had held that a book is addressed to any reader who proves worthy of it.', 'But, that night, I told Frank that I would dedicate The Fountainhead to him because he had saved it.', "And one of my happiest moments, about two years later, was given to me by the look on his face when he came home, one day, and saw 

In [0]:
#FOR TESTING
list_str = ['RAJU WENT TO RIVER BANK.', 'HE STARTED PLAYING FOOTBALL.']

In [0]:
import urllib
import ast
from bs4 import BeautifulSoup
import penman
import re
from py2neo import Graph, Node, Relationship

def save_to_neo4j(list_triples):
  g = Graph('http://localhost:7474/db/data', user='neo4j', password='root')
  for values in list_triples:
    tx = g.begin()
    for ele in values:
      print("\n")
      print(ele)
      if ele[1] == 'instance':
        d = g.run("MATCH (a:Concept) WHERE a.value={b} RETURN a", b=ele[2])
        list_d = list(d)
        if len(list_d) > 0:
          start_node = list_d[0]['a'] #
          start_node['label'] = ele[0] # return the updated value
          neo4j_transaction = g.begin()
          neo4j_transaction.push(start_node)
          neo4j_transaction.commit()
        else:
          neo4j_transaction = g.begin()
          start_node = Node("Concept", value=ele[2], label=ele[0], location=None, time=None, activity=None)
          neo4j_transaction.create(start_node)
          neo4j_transaction.commit()
      else:
        d = g.run("MATCH (a:Concept) WHERE a.label={b} RETURN a", b=ele[0])
        list_d = list(d)
        if len(list_d) > 0:
          start_node = list_d[0]['a'] #
        else:
          print("No node")

        #print(start_node['value'])
        d = g.run("MATCH (a:Concept) WHERE a.label={b} RETURN a", b=ele[2])
        list_d = list(d)
        if len(list_d) > 0:
          end_node = list_d[0]['a'] #
        else:
          print("No node")

        #print(end_node['value'])
        relation = ele[1]
        #print(relation)

        relations = g.match((start_node, end_node), r_type=relation)
        existing_relation = list(relations)

        if len(existing_relation) == 0:
          #print("No Relationship exists")
          neo4j_transaction = g.begin()
          ab = Relationship(start_node, relation, end_node)
          neo4j_transaction.create(ab)
          neo4j_transaction.commit()
          #print(g.exists(ab))
          
    for ele in values:
      #print("\n")
      #print(ele)
      if ele[1] == 'instance':
        d = g.run("MATCH (a:Concept) WHERE a.value={b} RETURN a", b=ele[2])
        list_d = list(d)
        print(list_d)
        if len(list_d) > 0:
          node = list_d[0]['a'] #
          node['label'] = None # return the updated value
          neo4j_transaction = g.begin()
          neo4j_transaction.push(node)
          neo4j_transaction.commit()

def sentence_to_amr_triplets(str_list):
  list_triples = []
  for each in str_list:
    #print(each)
    data = result = re.sub('/[^,a-zA-Z0-9_-]+', '', each)
    print(data)
    url_code = urllib.parse.quote_plus(data)
    #print(url_code)
    try:
      sock = urllib.request.urlopen("http://bollin.inf.ed.ac.uk:9010/amreager?lang=en&sent="+url_code) 
      htmlSource = sock.read().decode('gbk')                            
      sock.close()                                        
      #print (htmlSource)
      x = ast.literal_eval(htmlSource)
      #print(x)

      list_text =x['graph'].split('<br/>')
      del list_text[0:2]
      list_text

      str1 = ''.join(list_text)


      s = str1
      soup = BeautifulSoup(s)
      texts = soup.find_all(text=True)
      for t in texts:
         newtext = t.replace("&nbsp",'')
         t.replace_with(newtext)
      #print(newtext)

      pen_man = penman.decode(newtext)
      triples_list = pen_man.triples()

      list_triples.append(triples_list)
    
    except urllib.error.URLError as e:
      print(e)
    
  return list_triples

In [0]:
list_penmen = sentence_to_amr_triplets(list_str)
#print(list_penmen)

RAJU WENT TO RIVER BANK.
HE STARTED PLAYING FOOTBALL.


In [0]:
save_to_neo4j(list_penmen)

In [0]:
list_penmen

[[Triple(source='v3', relation='instance', target='go-01'),
  Triple(source='v1', relation='instance', target='person'),
  Triple(source='v2', relation='instance', target='name'),
  Triple(source='v5', relation='instance', target='bank'),
  Triple(source='v4', relation='instance', target='river'),
  Triple(source='v3', relation='ARG1', target='v1'),
  Triple(source='v1', relation='name', target='v2'),
  Triple(source='v2', relation='op1', target='"RAJU"'),
  Triple(source='v1', relation='wiki', target='"RAJU"'),
  Triple(source='v3', relation='ARG1', target='v5'),
  Triple(source='v5', relation='mod', target='v4')],
 [Triple(source='v1', relation='instance', target='start-01'),
  Triple(source='v2', relation='instance', target='play-01'),
  Triple(source='v3', relation='instance', target='football'),
  Triple(source='v1', relation='ARG1', target='v2'),
  Triple(source='v2', relation='ARG1', target='v3')]]

In [0]:
from py2neo import NodeMatcher

In [0]:
g = Graph('http://localhost:7474/db/data', user='neo4j', password='root')
matcher = NodeMatcher(g)

In [0]:
list(matcher.match("Concept"))
#matcher.match("Concept").first()
#list(matcher.match("Concept", value__exact="man"))
#list(matcher.match("Concept").where("_.value =~ 'p.*'").order_by("_.value").limit(5))

In [0]:
from py2neo import RelationshipMatcher

In [0]:
rel_match = RelationshipMatcher(g)

In [0]:
list(rel_match.match(nodes=None, r_type=None))

[(_0)-[:op2 {}]->(_300),
 (_0)-[:op2 {}]->(_1),
 (_0)-[:ARG1 {}]->(_1),
 (_0)-[:ARG1 {}]->(_156),
 (_0)-[:time {}]->(_125),
 (_0)-[:ARG0 {}]->(_180),
 (_0)-[:ARG2 {}]->(_1),
 (_0)-[:time {}]->(_108),
 (_0)-[:ARG1 {}]->(_103),
 (_0)-[:ARG1 {}]->(_46),
 (_0)-[:ARG2 {}]->(_3),
 (_0)-[:wiki {}]->(_2),
 (_0)-[:op1 {}]->(_2),
 (_0)-[:name {}]->(_2),
 (_0)-[:ARG2 {}]->(_2),
 (_0)-[:ARG1 {}]->(_2),
 (_0)-[:ARG0 {}]->(_1),
 (_1)-[:op1 {}]->(_220),
 (_2)-[:mod {}]->(_425),
 (_3)-[:mod {}]->(_107),
 (_3)-[:wiki {}]->(_20),
 (_3)-[:name {}]->(_20),
 (_20)-[:op2 {}]->(_20),
 (_20)-[:op1 {}]->(_20),
 (_21)-[:ARG0 {}]->(_25),
 (_21)-[:location {}]->(_100),
 (_21)-[:ARG1 {}]->(_26),
 (_21)-[:ARG0 {}]->(_21),
 (_21)-[:ARG1 {}]->(_40),
 (_21)-[:ARG1 {}]->(_22),
 (_21)-[:ARG0 {}]->(_20),
 (_21)-[:polarity {}]->(_20),
 (_22)-[:frequency {}]->(_23),
 (_23)-[:degree {}]->(_24),
 (_25)-[:ARG0 {}]->(_25),
 (_25)-[:ARG1 {}]->(_430),
 (_25)-[:mode {}]->(_44),
 (_25)-[:ARG0 {}]->(_1),
 (_25)-[:ARG1 {}]->(_44),
 